In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import ssl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import sys
sys.path.append('../')
from fun.models import *
sns.set_style("whitegrid")
from torch.utils.data import random_split
from tqdm import tqdm
import pandas as pd
torch.manual_seed(42)
np.random.seed(42)



In [ ]:

transform = transforms.ToTensor()

dataset = torchvision.datasets.CIFAR10(
    root="../data",
    train=True,
    download=True,
    transform=transform  
)


torch.manual_seed(42)
valid_size = int(0.1 * len(dataset))
train_size = len(dataset) - valid_size


train_ds, valid_ds = random_split(dataset, [train_size, valid_size])
len(train_ds), len(valid_ds)
    

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

def train(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    train_loss, correct = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    train_loss /= num_batches
    correct /= size
    print(f" Train Accuracy: {(100*correct):>0.1f}%, Train Avg loss {train_loss:>8f} \n")
    return correct, train_loss


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    print(f"Test Accuracy: {(100*correct):>0.1f}%, Test Avg loss: {test_loss:>8f} \n")
    return correct, test_loss
    

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:


class Net_dropout(nn.Module):
    def __init__(self, dropout_rate):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 5, padding=2)
        self.batch1 = nn.BatchNorm2d(8)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 5, padding=2)
        self.batch2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 10)

        self.drop1 = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.conv1(x)
        x = self.batch1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.drop1(x)

        x = self.conv2(x)
        x = self.batch2(x)
        x = F.relu(x)
        x = self.drop1(x)
        x = self.pool(x)

        x = torch.flatten(x, 1)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


In [ ]:
#------------------------------------------------------------------
#--------------------- DO PUSZCZENIA ------------------------------
#------------------------------------------------------------------
epochs = 30
batch_size = 8
weight_decay = [0.01, 0.05, 0.1, 0.2, 0.5]
lr = 1e-3
data = np.array([[0]*len(weight_decay)]*epochs)
wd_train_acc_history = pd.DataFrame(data, columns = weight_decay)
wd_test_acc_history = pd.DataFrame(data, columns = weight_decay)
wd_train_loss_history  = pd.DataFrame(data, columns = weight_decay)
wd_test_loss_history  = pd.DataFrame(data, columns = weight_decay)

train_dataloader = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True
    )
test_dataloader = DataLoader(
    valid_ds, 
    batch_size=batch_size,
    shuffle=False
    )

for wd in weight_decay:
    print(f"Batch size: {batch_size} \n")    
    

    net = Net_dropout(dropout_rate=0.25).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(net.parameters(), lr=lr, weight_decay=wd)
    
    for t in range(epochs):

        print(f"Epoch {t+1}\n-------------------------------")
        train_acc, train_loss = train(train_dataloader, net, criterion, optimizer)
        test_acc, test_loss = test(test_dataloader, net, criterion)
        wd_train_acc_history.loc[t,wd] = train_acc        
        wd_test_acc_history.loc[t,wd] = test_acc
        wd_train_loss_history.loc[t,wd] = train_loss
        wd_test_loss_history.loc[t,wd] = test_loss

wd_train_acc_history.to_csv("wd_train_acc_history.csv")
!cp wd_train_acc_history.csv "drive/My Drive/DL/"

wd_test_acc_history.to_csv("wd_test_acc_history.csv")
!cp wd_test_acc_history.csv "drive/My Drive/DL/

wd_train_loss_history.to_csv("wd_train_loss_history.csv")
!cp wd_train_loss_history.csv "drive/My Drive/DL/

wd_test_loss_history.to_csv("wd_test_loss_history.csv")
!cp wd_test_loss_history.csv "drive/My Drive/DL/
